In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

In [3]:
import sqlite3
import pandas as pd
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [4]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [5]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [7]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

LR = LinearRegression().fit(X_train, y_train)
PR = LR.predict(X_test)
R2 = r2_score(y_test, PR)
R2

0.6687594935356332

In [8]:
R = Ridge().fit(X_train, y_train)
PR = R.predict(X_test)
R2 = r2_score(y_test, PR)
R2


0.6662221670168521

In [9]:
L = Lasso().fit(X_train, y_train) 
PR = L.predict(X_test)
R2 = r2_score(y_test, PR)
R2

0.6671453631686304

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
data =[]
for i in range(-5,5):
    data.append(10**i)
GS = GridSearchCV(R, {'alpha':data}).fit(X_train,y_train)
LS = GridSearchCV(L, {'alpha':data}).fit(X_train,y_train)

In [11]:
GS.best_params_
LS.best_params_

{'alpha': 1e-05}

In [12]:
RC = RidgeCV(data).fit(X_train, y_train)
r2_score(y_test, RC.predict(X_test))



0.668751010992999

In [13]:
LC = LassoCV().fit(X_train, y_train)
r2_score(y_test, LC.predict(X_test))


0.6706431115795963

Результаты полученные с использованием LassoCV показали лучшее качество


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
RC = Pipeline ([('GS', StandardScaler()), 
                    ('model', RidgeCV())])
LC = Pipeline([('GS', StandardScaler()), 
                     ('model', LassoCV())])
RC.fit(X_train, y_train)
LC.fit(X_train, y_train)
print(r2_score(y_test, RC.predict(X_test)))
print(r2_score(y_test, LC.predict(X_test)))


0.6659677905050388
0.6683883969336302


Результат улучшился

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [16]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
df = pd.DataFrame(poly.fit_transform(X))
df


,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,...,87616.0,4528.8,117482.40,1474.08,234.09,6072.570,76.194,157529.6100,1976.5620,24.8004
1,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,...,58564.0,4307.6,96049.80,2211.88,316.84,7064.820,162.692,157529.6100,3627.6660,83.5396
2,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,...,58564.0,4307.6,95064.86,975.26,316.84,6992.374,71.734,154315.4089,1583.1049,16.2409
3,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,...,49284.0,4151.4,87607.86,652.68,349.69,7379.581,54.978,155732.8369,1160.2122,8.6436
4,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,...,49284.0,4151.4,88111.80,1183.26,349.69,7422.030,99.671,157529.6100,2115.4770,28.4089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,1.0,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,...,74529.0,5733.0,107013.27,2639.91,441.00,8231.790,203.070,153656.1601,3790.5433,93.5089
502,1.0,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,...,74529.0,5733.0,108353.70,2478.84,441.00,8334.900,190.680,157529.6100,3603.8520,82.4464
503,1.0,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,...,74529.0,5733.0,108353.70,1539.72,441.00,8334.900,118.440,157529.6100,2238.5160,31.8096
504,1.0,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,...,74529.0,5733.0,107411.85,1769.04,441.00,8262.450,136.080,154802.9025,2549.5560,41.9904


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size =2, random_state=42)
R = Ridge().fit(X_train, y_train)
L = Lasso().fit(X_train, y_train)
print(r2_score(y_test, R.predict(X_test)))
print(r2_score(y_test, L.predict(X_test)))


0.6167961299626735
0.5861967202158876


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.967e+03, tolerance: 4.262e+00
  model = cd_fast.enet_coordinate_descent(


Видно увеличение качества обучения

http://archive.ics.uci.edu/ml/datasets/Adult

In [18]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [19]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [20]:
df = data.columns
X = data[df[:-1]]
y = data[df[-1:]].applymap(lambda x: 0 if str(x) == "<=50K" else 1)
y.head(10)


,14
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,1
8,1
9,1


9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [21]:
from sklearn.impute import SimpleImputer
most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(most_frequent.fit_transform(X))


10. Выберите колонки с числовыми и категориальными переменными.

In [22]:
num = X.select_dtypes(include=['int64', 'float64']).columns
cat = X.select_dtypes(include=['object']).columns
print(num, cat)

Int64Index([0, 2, 4, 10, 11, 12], dtype='int64') Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')


11.Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).


In [23]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
num_trans = Pipeline(steps=[
    ('MS', MinMaxScaler())])
cat_trans = Pipeline(steps=[
    ('OE', OneHotEncoder())])
pre = ColumnTransformer(
    transformers=[
        ('num', num_trans, num),
        ('cat', cat_trans, cat)
        ])
t_pre = pre.fit_transform(X)


12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [24]:
from sklearn.metrics import accuracy_score as acc, f1_score as f1
from sklearn.linear_model import LinearRegression as LR
import time
import warnings
warnings.filterwarnings("ignore")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
print(acc(y, y_pred=y),
f1(y, y_pred=y))

1.0 1.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [25]:
from sklearn.model_selection import cross_val_score as cr
from sklearn.linear_model import LogisticRegression as LG
from sklearn.svm import SVC as SVC, LinearSVC as LS

mod = LG().fit(t_pre,y)
accuracy = cr(mod, t_pre, y, scoring='accuracy', cv=5)
f1_score = cr(mod, t_pre, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8567772317772318 0.6661894535910284


In [26]:
mod_1 = LS().fit(t_pre,y)
accuracy = cr(mod_1, t_pre, y, scoring='accuracy', cv=5)
f1_score = cr(mod_1, t_pre, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8572891072891073 0.6645813282001924


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [27]:
Nan = X.replace(['?'], np.NaN)
num_trans = Pipeline(steps=[
    ('MS', MinMaxScaler())])
cat_trans = Pipeline(steps=[
    ('OE', OneHotEncoder())])
pre = ColumnTransformer(
    transformers=[
        ('num', num_trans, num),
        ('cat', cat_trans, cat)
        ])
t_pre_Nan = pre.fit_transform(Nan)


In [28]:
mod = LG().fit(t_pre_Nan,y)
accuracy = cr(mod, t_pre_Nan, y, scoring='accuracy', cv=5)
f1_score = cr(mod, t_pre_Nan, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8572891072891073 0.6674618320610687


In [29]:
mod_1 = LS().fit(t_pre_Nan,y)
accuracy = cr(mod_1, t_pre_Nan, y, scoring='accuracy', cv=5)
f1_score = cr(mod_1, t_pre_Nan, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8572891072891073 0.6645813282001924


Результат улучшился

17.Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [32]:
from sklearn.ensemble import RandomForestClassifier as RC, GradientBoostingClassifier as GC
mod = RC().fit(t_pre_Nan, y)
accuracy = cr(mod, t_pre_Nan, y, scoring='accuracy', cv=5)
f1_score = cr(mod, t_pre_Nan, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8578009828009828 0.6767161410018552


In [33]:
mod_1 = GC().fit(t_pre_Nan, y)
accuracy = cr(mod, t_pre_Nan, y, scoring='accuracy', cv=5)
f1_score = cr(mod, t_pre_Nan, y, scoring='f1', cv=5)
print(accuracy.max(), f1_score.max())

0.8549344799344799 0.6720704029643354


По результату модель GradientBoostingClassifier самая лучшая.